# Meeting
* Members skills presentation
## What area is covered by sugar cane?
* Clouds? -- SCL help (1 person)
* Preprocessing
* Clustering
   + [Overview different algorithms](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html)
   + [Performance evaluation](https://stats.stackexchange.com/questions/21807/evaluation-measures-of-goodness-or-validity-of-clustering-without-having-truth)
   + Performance in literature
* Time-depending ideas?
* Another approach?
* Expert (1 person)
* Time Series? (can be the expert)

In [19]:
import os
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import xarray
import datetime

In [2]:
# read in the sentinel 2 data
s2_data = xarray.open_dataset('cali_training.nc')
s2_data

<xarray.Dataset>
Dimensions:      (band: 10, date: 26, x: 994, y: 940)
Coordinates:
  * band         (band) object 'B02' 'B03' 'B04' 'B05' ... 'B8A' 'B11' 'B12'
  * date         (date) datetime64[ns] 2016-12-18 2017-02-16 ... 2019-01-22
Dimensions without coordinates: x, y
Data variables:
    reflectance  (y, x, band, date) uint16 ...
    mask         (y, x) bool ...
    SCL          (y, x, date) uint16 ...
Attributes:
    transform:  [ 1.0000e+01  0.0000e+00  3.3092e+05  0.0000e+00 -1.0000e+01 ...
    crs:        +init=epsg:32618

In [25]:
band_dict = {k: v for v,k in enumerate(s2_data.band.values)}
date_dict = {str(k)[:10]: v for v,k in enumerate(s2_data.date.values)}

In [34]:
w, h = s2_data.reflectance.isel(date=0, band=0).values.shape

def create_dataset(obj, d):
    l, = s2_data.band.values.shape
    return np.concatenate([obj.isel(date=d, band=b).values.reshape(w, h, 1) for b in range(l)], axis=-1)

In [35]:
def add_feature(X, fea, ax=2):
    return np.concatenate((X, fea), axis=ax)

In [36]:
X_refl_0 = create_dataset(s2_data.reflectance, 0)/10000

In [38]:
r = X_refl_0[:,:,band_dict['B04']] 
ir = X_refl_0[:,:,band_dict['B08']]
add_feature(X_refl_0, ((ir-r)/(ir+r)).reshape(w, h, 1))

array([[[0.039     , 0.0667    , 0.0458    , ..., 0.2364    ,
         0.1192    , 0.75229854],
        [0.0379    , 0.0649    , 0.039     , ..., 0.2411    ,
         0.125     , 0.79639781],
        [0.0392    , 0.0645    , 0.0429    , ..., 0.2502    ,
         0.1353    , 0.79027133],
        ...,
        [0.0426    , 0.0698    , 0.0514    , ..., 0.1665    ,
         0.0756    , 0.70803749],
        [0.0419    , 0.068     , 0.0501    , ..., 0.1492    ,
         0.0641    , 0.71702909],
        [0.0417    , 0.0674    , 0.0479    , ..., 0.1419    ,
         0.0595    , 0.73307328]],

       [[0.0395    , 0.0673    , 0.049     , ..., 0.2459    ,
         0.1312    , 0.71353405],
        [0.041     , 0.0661    , 0.045     , ..., 0.2545    ,
         0.1415    , 0.74761638],
        [0.044     , 0.0709    , 0.0544    , ..., 0.27      ,
         0.1592    , 0.7174026 ],
        ...,
        [0.0415    , 0.0652    , 0.0474    , ..., 0.1645    ,
         0.0728    , 0.73355818],
        [0.0